# Reliability of borrowers

The credit department of a bank is the client for this project, and they are seeking to investigate the potential impact of a customer's marital status and number of children on their ability to repay loans promptly. The bank will utilize customer payment capability statistics as their input data.

The findings of this research will be incorporated into the development of a **credit scoring** model, which is a specialized system designed to evaluate a potential borrower's ability to repay a loan to the bank.

## Step 1. Open the dataset and review the info

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pymystem3 import Mystem
from collections import Counter

m = Mystem()

Installing mystem to /root/.local/bin/mystem from http://download.cdn.yandex.net/mystem/mystem-3.1-linux-64bit.tar.gz


In [2]:
try:
    df = pd.read_csv("/datasets/data.csv")
except FileNotFoundError:
    try:
        df = pd.read_csv("data.csv")
    except FileNotFoundError:
        try:
            df = pd.read_csv(r"data.csv")
        except FileNotFoundError:
            print("File not found")

In [3]:
print("Shape of dataframe: {}\n".format(df.shape))
print("-" * 26)
print("Missing values:\n")
print(df.isna().mean().sort_values())
print("-" * 26)
print("Duplicates: {}\n".format(df.duplicated().sum()))
print("-" * 26)
print("Data type:\n")
print(df.dtypes)
display(df.describe())
display(df.head())

Shape of dataframe: (21525, 12)

--------------------------
Missing values:

children            0.000000
dob_years           0.000000
education           0.000000
education_id        0.000000
family_status       0.000000
family_status_id    0.000000
gender              0.000000
income_type         0.000000
debt                0.000000
purpose             0.000000
days_employed       0.100999
total_income        0.100999
dtype: float64
--------------------------
Duplicates: 54

--------------------------
Data type:

children              int64
days_employed       float64
dob_years             int64
education            object
education_id          int64
family_status        object
family_status_id      int64
gender               object
income_type          object
debt                  int64
total_income        float64
purpose              object
dtype: object


,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [4]:
print(f"Unique values in DOB: {np.array(sorted(df['dob_years'].unique()))}")

Unique values in DOB: [ 0 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41
 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65
 66 67 68 69 70 71 72 73 74 75]


In [5]:
df["education"].value_counts()

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

In [6]:
df["gender"].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

**Conclusion**

- `children` and `days_employed` contains negative values.
- `dob_years` contains "0" as values.
- `gender` contains "XNA" value.
- There's a different case in values in `education` column.

## Step 2. Data preprocessing

### Missing values processing

Replacing NaN values in `days_employed` with "0":

In [7]:
df["days_employed"].isna().sum()
df["days_employed"] = df["days_employed"].fillna(0)
df["days_employed"].isna().sum()

0

Checking the NaNs in `total_income` grouped by type of job:

In [8]:
df_no_income = df[df["total_income"].isna() == True]
df_no_income["income_type"].value_counts()

сотрудник          1105
компаньон           508
пенсионер           413
госслужащий         147
предприниматель       1
Name: income_type, dtype: int64

The amount of NaNs for `income_type` is defined for different type of job. Since there's no proved information about income, the NaNs will be filled with median value.

In [9]:
df["total_income"] = df["total_income"].fillna(
    df.groupby("income_type")["total_income"].transform("median")
)
df.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

**Conclusion**

There is no straight dependency between missing data in `days_employed` and `total_income`, but leaving 2174 rows with NaN values is not the solution. So missing values in `total_income` were replaced with median grouped by each type of job (`income_type` column).

### Data fixing

Converting float to integer type. In my point of view, we can get rid of decimals in `total_income` this won't affect on result.

In [10]:
df["total_income"] = df["total_income"].astype(int)
df["days_employed"] = df["days_employed"].astype(int)

Also let's check the anomalies for negative values in `children` and `days_employed`, zeros in `dob_years` and "XNA" in `gender` columns.

#### Children

In [11]:
df["children"].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

Amount of "negative" children:

In [12]:
df[df["children"] == -1]["children"].count()

47

Highly likely that 20 kids is a typo, so fixing this too:

In [13]:
df["children"] = df["children"].replace([20, -1], [2, 1])
df["children"].value_counts()

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

#### Days employed

In [14]:
df["days_employed"].describe()

count     21525.000000
mean      56679.163252
std      134870.418456
min      -18388.000000
25%       -2518.000000
50%        -982.000000
75%           0.000000
max      401755.000000
Name: days_employed, dtype: float64

Max value of 401755 looks very suspicious, marking this as "garbage in" and leaving it until following steps to prevent "garbage out".

#### Age (in years)

In [15]:
df["dob_years"].describe()

count    21525.000000
mean        43.293380
std         12.574584
min          0.000000
25%         33.000000
50%         42.000000
75%         53.000000
max         75.000000
Name: dob_years, dtype: float64

There's a value of "0", I should check these:

In [16]:
df[df["dob_years"] == 0]["dob_years"].count()
df[df["dob_years"] == 0].sample(3)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
14608,0,-1641,0,среднее,1,гражданский брак,1,M,сотрудник,1,129945,сыграть свадьбу
18732,2,-986,0,высшее,0,гражданский брак,1,F,компаньон,0,287328,образование
2941,2,-2161,0,среднее,1,женат / замужем,0,F,сотрудник,0,199739,на покупку автомобиля


At this step, I have decided to leave the data "as is" and check back during the final step to determine whether it makes sense to replace the zeros with the mean value and group them based on income type, sex, or family status.

#### Education

Let's see how many unique values here:

In [17]:
len(df["education"].unique())

15

In [18]:
df["education"] = df["education"].str.lower()
df["education"].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

In [19]:
df["education_id"].unique()

array([0, 1, 2, 3, 4])

All entries were transferred to small case, and now we have 5 unique values, which amount is the same as amount in `education_id`.

#### Family status

There's a capital letter in "Не женат / не замужем", let's fix this to small case:

In [20]:
df["family_status"].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

In [21]:
df["family_status"] = df["family_status"].str.lower()
df["family_status"].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'не женат / не замужем'], dtype=object)

#### Gender

It was defined that `gender` contains "XNA" let's locate the row:

In [22]:
df[df["gender"] == "XNA"].squeeze()

children                               0
days_employed                      -2358
dob_years                             24
education            неоконченное высшее
education_id                           2
family_status           гражданский брак
family_status_id                       1
gender                               XNA
income_type                    компаньон
debt                                   0
total_income                      203905
purpose             покупка недвижимости
Name: 10701, dtype: object

### Duplicates

The amount of duplicates:

In [23]:
df.duplicated().sum()

71

In [24]:
df = df.drop_duplicates().reset_index(drop=True)
df.duplicated().sum()

0

**Conclusion**

Possible reason of duplicates is mistakenly doubled data.

### Lemmatization

Let's check how many unique purposes we have:

In [25]:
df["purpose"].nunique()

38

In [26]:
purpose_lemmas_join = "; ".join(df["purpose"].unique())
purpose_lemmas_join

'покупка жилья; приобретение автомобиля; дополнительное образование; сыграть свадьбу; операции с жильем; образование; на проведение свадьбы; покупка жилья для семьи; покупка недвижимости; покупка коммерческой недвижимости; покупка жилой недвижимости; строительство собственной недвижимости; недвижимость; строительство недвижимости; на покупку подержанного автомобиля; на покупку своего автомобиля; операции с коммерческой недвижимостью; строительство жилой недвижимости; жилье; операции со своей недвижимостью; автомобили; заняться образованием; сделка с подержанным автомобилем; получение образования; автомобиль; свадьба; получение дополнительного образования; покупка своего жилья; операции с недвижимостью; получение высшего образования; свой автомобиль; сделка с автомобилем; профильное образование; высшее образование; покупка жилья для сдачи; на покупку автомобиля; ремонт жилью; заняться высшим образованием'

In [27]:
lemmas = m.lemmatize(purpose_lemmas_join)
Counter(lemmas)

Counter({'покупка': 10,
         ' ': 59,
         'жилье': 7,
         '; ': 37,
         'приобретение': 1,
         'автомобиль': 9,
         'дополнительный': 2,
         'образование': 9,
         'сыграть': 1,
         'свадьба': 3,
         'операция': 4,
         'с': 5,
         'на': 4,
         'проведение': 1,
         'для': 2,
         'семья': 1,
         'недвижимость': 10,
         'коммерческий': 2,
         'жилой': 2,
         'строительство': 3,
         'собственный': 1,
         'подержать': 2,
         'свой': 4,
         'со': 1,
         'заниматься': 2,
         'сделка': 2,
         'получение': 3,
         'высокий': 3,
         'профильный': 1,
         'сдача': 1,
         'ремонт': 1,
         '\n': 1})

Main purposes:
- жилье/недвижимость (7/10);
- автомобиль (9);
- образование (9);
- свадьба (3)

In [28]:
def purpose_category(row):
    lem_purpose = m.lemmatize(row)
    if "автомобиль" in lem_purpose:
        return "автомобиль"
    if ("жилье" in lem_purpose) or ("недвижимость" in lem_purpose):
        return "жильё, недвижимость"
    if "образование" in lem_purpose:
        return "образование"
    if "свадьба" in lem_purpose:
        return "свадьба"
    return "иное"

In [29]:
df["purpose_category"] = df["purpose"].apply(purpose_category)
df.tail(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category
21439,2,0,28,среднее,1,женат / замужем,0,F,сотрудник,0,142594,приобретение автомобиля,автомобиль
21440,0,-612,29,высшее,0,гражданский брак,1,F,сотрудник,1,140068,покупка жилья для сдачи,"жильё, недвижимость"
21441,0,-165,26,высшее,0,не женат / не замужем,4,M,компаньон,0,147301,получение дополнительного образования,образование
21442,0,-1166,35,среднее,1,женат / замужем,0,F,сотрудник,0,250986,покупка жилья,"жильё, недвижимость"
21443,0,-280,27,неоконченное высшее,2,не женат / не замужем,4,M,компаньон,0,355988,строительство недвижимости,"жильё, недвижимость"
21444,1,-467,28,среднее,1,женат / замужем,0,F,сотрудник,1,109486,заняться образованием,образование
21445,0,-914,42,высшее,0,женат / замужем,0,F,компаньон,0,322807,покупка своего жилья,"жильё, недвижимость"
21446,0,-404,42,высшее,0,гражданский брак,1,F,компаньон,0,178059,на покупку своего автомобиля,автомобиль
21447,0,373995,59,среднее,1,женат / замужем,0,F,пенсионер,0,153864,сделка с автомобилем,автомобиль
21448,1,-2351,37,ученая степень,4,в разводе,3,M,сотрудник,0,115949,покупка коммерческой недвижимости,"жильё, недвижимость"


**Conclusion**

Lemmatization and categorization of data results four main categories for purposes:
- Real estate (living and commercial)
- Car purchasing
- Educational loan
- Wedding organizing

### Categorization of data

Data categorization by number of children:

In [30]:
def children_category(children):
    """Define if there's a children or not"""
    if children < 1:
        return "детей нет"
    if children < 2:
        return "один ребенок"
    return "многодетная семья"


df["children_category"] = df["children"].apply(children_category)

In [31]:
edu_cat = df.loc[1418:1427, ["education", "education_id"]]
df["education"].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

In [32]:
fam_cat = df.loc[2000:2005, ["family_status", "family_status_id"]]
df["family_status"].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'не женат / не замужем'], dtype=object)

Adding three categroies of income (low, middle and high income):

In [33]:
mean_total_income = df["total_income"].mean()

def total_income_category(total_income, mean_income):
    """Define category income"""
    if total_income <= mean_income / 2:
        return "низкий уровень дохода"
    if total_income <= mean_income:
        return "средний уровень дохода"
    return "высокий уровень дохода"

df["total_income_category"] = df["total_income"].apply(
    total_income_category, mean_income=mean_total_income)

Print out categories of clients who has loan:

In [34]:
debtors_income_type = df.groupby(
    "income_type")["debt"].mean().sort_values(ascending=False)
debtors_income_type

income_type
в декрете          1.000000
безработный        0.500000
сотрудник          0.095724
компаньон          0.074045
госслужащий        0.059025
пенсионер          0.056412
предприниматель    0.000000
студент            0.000000
Name: debt, dtype: float64

**Conclusion**

- It was found out that biggest debtors are those who are on maternity leave (в декрете) and unemployed clients (берзаботный). Least of all, the debts are retired (пенсионер), business owners (предприниматель) and students (студент).
- I made the categorization by purpose of loans previously on step 2.4

## Step 3. Questions and answers

- Is there any relation between having a children and repayment of loan in time?

Categorizing clients by number of children:

In [35]:
children_pivot = df.pivot_table(
    index="children_category",
    columns="debt",
    values="family_status_id",
    aggfunc="count",
)
children_pivot["ratio"] = (
    children_pivot[1] / (children_pivot[0] + children_pivot[1])
).apply(lambda x: "{:.2%}".format(x))
children_pivot.sort_values(by="ratio", ascending=False)

debt,0,1,ratio
children_category,,,
многодетная семья,2275,233,9.29%
один ребенок,4410,445,9.17%
детей нет,13028,1063,7.54%


**Conclusion**

The allocation between childless clients and large families showed the following relationship: the loan repayment rate for borrowers without children is higher than for clients from large families. Childless clients are 1.75% more reliable.

- Is there a relation between marital status and repayment of loan during the term?

Categorizing clients by family status:

In [36]:
family_pivot = df.pivot_table(
    index="family_status",
    columns="debt",
    values="family_status_id",
    aggfunc="count"
)
family_pivot["ratio"] = (
    family_pivot[1] / (family_pivot[0] + family_pivot[1])
).apply(lambda x: "{:.2%}".format(x))
family_pivot.sort_values(by="ratio", ascending=False)

debt,0,1,ratio
family_status,,,
не женат / не замужем,2536,274,9.75%
гражданский брак,3763,388,9.35%
женат / замужем,11408,931,7.55%
в разводе,1110,85,7.11%
вдовец / вдова,896,63,6.57%


**Conclusion**

The pivot table above shows that the most unreliable borrowers are clients with a marital status of single (не женат / не замужем). In opposite, the most reliable are those who are divorced (в разводе) or have lost spouses (вдовец / вдова).

- Is there a relation between the type of income and repayment of the loan in time?

Categorizing clients by income:

In [37]:
income_pivot = df.pivot_table(
    index="total_income_category",
    columns="debt",
    values="purpose",
    aggfunc="count"
)
income_pivot["ratio"] = (
    income_pivot[1] / (income_pivot[0] + income_pivot[1])
).apply(lambda x: "{:.2%}".format(x))
income_pivot.sort_values(by="ratio", ascending=False)

debt,0,1,ratio
total_income_category,,,
средний уровень дохода,9804,920,8.58%
низкий уровень дохода,2362,197,7.70%
высокий уровень дохода,7547,624,7.64%


**Conclusion**

Clients with high-income (высокий уровень дохода) have the highest rate of loan fulfillment, clients with low-income (низкий уровень дохода) almost next to them. The most unreliable clients are "middle class" (средний уровень дохода).

- How do different purposes of the loan affect its repayment in time?

Categorizing purposes of loan:

In [38]:
purpose_pivot = df.pivot_table(
    index="purpose_category",
    columns="debt",
    values="purpose",
    aggfunc="count"
)
purpose_pivot["ratio"] = (
    purpose_pivot[1] / (purpose_pivot[0] + purpose_pivot[1])
).apply(lambda x: "{:.2%}".format(x))
purpose_pivot.sort_values(by="ratio", ascending=False)

debt,0,1,ratio
purpose_category,,,
автомобиль,3903,403,9.36%
образование,3643,370,9.22%
свадьба,2138,186,8.00%
"жильё, недвижимость",10029,782,7.23%


**Conclusion**

Auto loan and car financing is the most risky type of loan. The difference between the most reliable category "real estate" (жильё, недвижимость) is 2.13%.

## Step 4. General conclusion

Putting together the previous conclusion, now we can describe three types of borrowers:
1. Reliable borrower: childless widower/widow (or divorced), low or high income. The purpose of the loan is real estate.
2. Unreliable borrower 1: a large family with an average income. The purpose of the loan is a car or education.
3. Unreliable borrower 2: civil married clients with an average income. The purpose of the loan is the same as above.